In [1]:
#pip install scikit-learn


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [3]:
#reading the files
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

In [4]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
df_fake['label'] = 0
df_true['label'] = 1

In [6]:
df_fake = df_fake.drop(columns = ['subject', 'date', 'title'])
df_true = df_true.drop(columns = ['subject', 'date', 'title'])

In [7]:
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.head()

,text,label
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [8]:
df_merge.columns

Index(['text', 'label'], dtype='object')

In [9]:
#checking if therr is some empty cells i.e. null values
df_merge.isnull().sum()

text     0
label    0
dtype: int64

In [10]:
def word_proc(text):
    '''it takes a text as input and performs various text preprocessing steps, such as converting the text to lowercase, removing square brackets and their content, 
    replacing non-word characters with spaces, removing URLs, removing HTML tags, removing punctuation, removing newline characters, and removing words with digits.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [11]:
df_merge["text"] = df_merge["text"].apply(word_proc)

In [12]:
x = df_merge["text"]
y = df_merge["label"]

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [15]:

# # Define the logistic sigmoid function
# def sigmoid(z):
#     return 1 / (1 + np.exp(-z))

# # Initialize learning rate and number of iterations
# learning_rate = 0.01
# num_iterations = 100

# # Initialize weights and bias
# n_features = xv_train.shape[1]  # Number of features in your TF-IDF matrix
# weights = np.zeros(n_features)
# bias = 0

# # Training the logistic regression model
# for i in range(num_iterations):
#     # Compute the linear combination of weights and features
#     z = np.dot(xv_train, weights) + bias
#     # Apply the sigmoid function to get the predicted probabilities
#     predictions = sigmoid(z)
    
#     # Compute the gradient of the cost function with respect to weights and bias
#     dw = (1 / len(xv_train)) * np.dot(xv_train.T, (predictions - y_train))
#     db = (1 / len(xv_train)) * np.sum(predictions - y_train)
    
#     # Update weights and bias
#     weights -= learning_rate * dw
#     bias -= learning_rate * db

# # Making predictions on the test data
# z_test = np.dot(xv_test, weights) + bias
# predictions_test = sigmoid(z_test)
# y_pred = (predictions_test >= 0.5).astype(int)

# # Evaluate the model (e.g., using accuracy or other metrics)
# accuracy = np.mean(y_pred == y_test)
# print("Accuracy:", accuracy)


In [16]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(xv_train,y_train)
pred_lr=LR.predict(xv_test)
print(LR.score(xv_test, y_test))

0.9871714922048997


In [17]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(word_proc) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)

    return print("Prediction: {} ".format(output_lable(pred_LR[0]) ))

In [24]:
news = str(input())
print('News is :')
print (news)
manual_testing(news)

#for true : Transgender people will be allowed for the first time to enlist in the U.S. military starting on Monday as ordered by federal courts, the Pentagon said on Friday, after President Donald Trump’s administration decided not to appeal rulings that blocked his transgender ban

News is :
Transgender people will be allowed for the first time to enlist in the U.S. military starting on Monday as ordered by federal courts, the Pentagon said on Friday, after President Donald Trump’s administration decided not to appeal rulings that blocked his transgender ban. Two federal appeals courts, one in Washington and one in Virginia, last week rejected the administration’s request to put on hold orders by lower court judges requiring the military to begin accepting transgender recruits on Jan. 1. A Justice Department official said the administration will not challenge those rulings. “The Department of Defense has announced that it will be releasing an independent study of these issues in the coming weeks. So rather than litigate this interim appeal before that occurs, the administration has decided to wait for DOD’s study and will continue to defend the president’s lawful authority in District Court in the meantime,” the official said, speaking on condition of anonymity. 

In [19]:
import pickle

In [20]:
# Save the trained model to a file
with open('fake_news_model.pkl', 'wb') as model_file:
    pickle.dump(LR, model_file)

# Save the vectorizer to a file
with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorization, vectorizer_file)